In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
from torch.distributions import Categorical
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader

from model import VAE_model
import metadata as md
from utilities import gumbel_softmax
from Architetture.VAE.discrete.myDatasets import Dataset_HFM
from train import train

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Utilizzo Apple Silicon GPU (MPS)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Utilizzo NVIDIA GPU (CUDA)")
else:
    device = torch.device("cpu")
    print("Utilizzo la CPU")

In [ ]:
torch.manual_seed(0)

torch.manual_seed(seed)
if device.type == "cuda": 
    torch.cuda.manual_seed(seed)
elif device.type == "mps": 
    torch.mps.manual_seed(seed)

In [2]:
import sys
print(sys.path)

['/opt/homebrew/Cellar/python@3.11/3.11.12_1/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.12_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.12_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/enricofrausin/Programmazione/Python_nn/.venv/lib/python3.11/site-packages', '/var/folders/bb/kqb36gjx6c51yhmhfvh767rr0000gn/T/tmpg_rzhtd4']


# MNIST

In [ ]:

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        './data/MNIST',
        train=True,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
    )

val_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        './data/MNIST',
        train=False,
        download=True,
        transform=transforms.ToTensor()
        ),
    batch_size=batch_size,
    shuffle=False,
    **kwargs
    )

# Train loop

In [ ]:
writer = SummaryWriter(log_dir='runs/discrete_VAE_Categorical/_1')

In [ ]:

my_model = VAE_model().to(device)
optimizer = optim.Adam(my_model.parameters(), lr=1e-3)
train(my_model, optimizer, epochs=15)

In [ ]:
if __name__ == '__main__':
    kwargs = {'num_workers': 0, 'pin_memory': False} if torch.backends.mps.is_available() else {}
    if torch.backends.mps.is_available():
        print("Running on Apple Silicon (MPS). num_workers set to 0 to avoid pickling issues.")
        kwargs = {'num_workers': 0, 'pin_memory': False} # pin_memory a False per MPS con num_workers=0


    # Inizializzazione del dataset e dei DataLoader
    dataset_HFM = Dataset_HFM(csv_file='data/feat_512_g_log2_numex_60000.csv',
                            root_dir='data')
    train_loader = DataLoader(
        dataset_HFM,
        batch_size=md.batch_size,
        shuffle=True,
        **kwargs
    )
    dataset_HFM_val = Dataset_HFM(csv_file='data/feat_512_g_log2_numex_10000.csv',
                                root_dir='data')
    val_loader = DataLoader(
        dataset_HFM_val, # Importante: usa dataset_HFM_val qui, non dataset_HFM
        batch_size=md.batch_size,
        shuffle=False,
        **kwargs
    )

    my_model = VAE_model()
    my_model.to(device)
    optimizer = optim.Adam(my_model.parameters(), lr=1e-3)

    # Chiamata alla funzione train
    train(my_model, optimizer, epochs=20)
